<img src="img/logo.png" align="right" style="width: 200px;"/> 
## Vorhersage von Patentklassifikation - WS 18/19

### Kooperationsseminar AIFB und ECON
---
Namen: Marco Seidler (1767526), Paolo Ramella-Ratin (1958685)
    
Betreuer: M.Sc. David Bälz , M.Sc. Tobias Weller

## Inhaltsverzeichnis

1. [Einleitung](#kapitel1)
2. [Theroretische Grundlagen](#kapitel2)
    1. [International Patent Classification](#kapitel2.1)
    2. [Automated Hierarchical Text Classification](#kapitel2.2)
    3. [Vergleichsmodelle und Evaluationsmetriken](#kapitel2.3)
3. [Implementierung](#kapitel3)
    1. [Daten](#kapitel3.1)
    2. [Preprocessing](#kapitel3.2)
    3. [Hierarchie-Implementierung](#kapitel3.3) 
    4. [Modelle](#kapitel3.4)
        1. [Baseline Modell: Flat, Decision Tree](#kapitel3.4.1)
        2. [Hierarchie Modell 1: LCPN](#kapitel3.4.2)
        3. [Hierarchie Modell 2: LCN](#kapitel3.4.3)
    5. [Ergebnisse](#kapitel3.5) 
4. [Limitationen](#kapitel4)
5. [Fazit](#kapitel5)
6. [Referenzen](#referenzen)

## 1. Einleitung <a name="kapitel1"></a>

Mit dem steigenden Bedarf am Schutz geistigen Eigentums und fortwährend komplexer werdender Technologien ist der Aufwand für Patent-Organisationen in den letzten Jahren enorm gestiegen. Weltweit stehen Patent-Organisationen vor der Aufgabe, eine stetig steigende Zahl an Patenten im Bezug auf ihre technischen Merkmale und Anwendungsgebiete zu klassifizieren. Diese Klassifikation wird meist in komplexen hierarchischen Strukturen, wie die International Patent Classification (IPC), vorgenommen. In diesen aus mehreren zehntausenden Kategorien und multiplen Ebenen bestehenden Hierarchien müssen Analysten einem Patent eine oder mehrere Kategorien zuweisen. Im Zeitalter von Big Data und mit den heutigen Möglichkeiten im Hinblick auf Rechenleistung und dem Stand der Forschung im Bereich Maschinelles Lernen liegt es Nahe, ein System zu entwickeln, welche die Aufgabe automatisiert übernehmen oder die Analysten bei Ihren Entscheidungen unterstützen kann.

Das Ziel dieser Seminararbeit ist nun, ein solches automatisiertes Klassifikationsystem zu entwickeln, welches unter Verwendung hierarchischer Informationen Patente mit möglichst hoher Treffsicherheit und befriedigender Laufzeit vornehmen kann. Dabei soll zunächst ein generelles Framework für schrittweise bzw. knotenbasierte hierarchische Text-Klassifikation implementiert werden, das mit verschiedenen Knotendesigns genutzt werden kann und auf dem die final implementierten Modelle aufbauen.

In den folgenden Teilen der Arbeit wird hierbei zunächst in [Kapitel 2](#kapitel2) auf den betrachteten Datensatz, das Automated Hierarchical Text Classification Problem und die dafür in Betracht kommenden Hierarchie-Designs eines entsprechenden Klassifikationssystems sowie die für die im Rahmen dieser Seminararbeit entwickelten Modelle herangezogenen Vergleichsmodelle und Evaluationsmetriken eingegangen. 
Anschließend wird in [Kapitel 3](#kapitel3) die Implementierung beschrieben und umgesetzt. Hierbei wird schrittweise durch die Auswahl der verwendeten Bestandteile der Patente, das Vorgehen beim Preprocessing der Daten, die Entwicklung des Hierarchie-Klassifikations-Frameworks sowie Implementierung von zwei expliziten Modellen geführt. Am Ende des Kapitels findet eine Darstellung und Bewertung der Modell-Performances anhand der eingeführten Evaluationsmetriken und Vergleichsmodelle statt. 
In [Kapitel 4](#kapitel4) werden die Limitationen und mögliche Anknüpfungsmöglichkeiten dieser Arbeit aufgezeigt, um anschließend in [Kapitel 5](#) die Seminararbeit und ihre Ergebnisse zusammenzufassen.

# 2. Theoretische Grundlagen <a name="kapitel2"></a>

In diesem Kapitel soll zunächst ein Verständnis für die IPC und dem ihr zugrunde liegenden Klassifikationsproblem vermittelt werden. Anschließend werden basierend darauf sowie auf bereits existierenden Veröffentlichungen Evaluationsmetriken abgeleitet, die später für die Bewertung der implementierten Modelle herangezogen werden.

### A. International Patent Classification <a name="kapitel2.1"></a>

Die IPC ist das weltweit meist verbreiteste Klassifikationssystem und wird sowohl von internationalen als auch von nationalen Organisationen genutzt um Patente und Patentanmeldungen zu ordnen. Um die Patente entsprechend ihrer technischen Merkmale und Anwendungsgebiete strukturiert einordnen zu können ist die IPC als Hierarchie aufgebaut, welche regelmäßig von Experten der World Intellectual Property Organization (WIPO) aktualisiert wird. Diese Hierarchie ist hierbei mit einem gerichteten azyklischen Graphen vergleichbar, bei dem jede Kategorie durch einen Knoten repräsentiert wird. Dabei werden untergeordnete Knoten immer präziser definiert als ihre übergeordneten Knoten. Entlang der fünf Hierarchie-Level ergibt sich so für jedes Patent ein oder mehrere IPC-Codes, die jeweils einen eindeutigen Pfad durch die Hierchie darstellen. Eine Veranschaulichung der Hierarchieebenen und den damit verbunden IPC-Codes ist in [Tabelle 1](#t1) dargestellt. 

|Kategorie<a name="t1"></a> |IPC-Code|Titel               | 
|:-------------------------:|:------:|:------------------:| 
|Section                    |A       |Human Necessities   |
|Class                      |A42     |Headwear            |
|Subclass                   |A42B    |Hats; Head Coverings|
|Main group                 |A42B1/00|Hats; Caps; Hoods   |
|Subgroup                   |A42B1/04|Soft caps; Hoods    |


### B. Automated Hierarchical Text Classification <a name="kapitel2.2"></a>

Der Klassifizierung von Patenten und Patentanmeldungen anhand textueller Informationen liegt das Automated Hierarchical Text Classification (AHTC) Problem zugrunde. Dieses Problem weist insbesondere zwei für diese Seminararbeit relevenate Merkmale auf: Zum einen ist es ein hierarchisches Einordnungsproblem und ermöglicht es so, das Problem in Teilprobleme entlang der Hierarchie aufzuteilen. Zum Anderen ist es ein Multi-Label-Problem, was bedeutet, dass ein Patent in mehr als eine Kategorie eingeordnet werden kann. Beide Merkmale sind für die spätere Entwicklung und Implementierung der Modelle elementar.

Um eine Klassifizierung auf einem speziellen Hierachie-Level zu erreichen lassen sich grundsätzlich zwei Vorgehensweisen unterscheiden: Die erste Vorgehensweise sieht dabei keine Verwendung von hierarchischen Informationen vor und ein "flacher" (flat) Klassifikator wird direkt auf das entsprechende Ziel-Hierarchie-Level trainiert. Bei der zweiten Vorgehensweise hingegen werden die sich aus der Hierarchie ergebenden Informationen genutzt. Im Zuge dieser Seminararbeit wird ein "flacher" Klassifikator als Baseline-Modell dienen, um später die entwickelten hierarchischen Klassifikationsmodelle vergleichen zu können. 

Nach Silla und Freitas [[2011](#ref1)] lassen sich bei hierarchischen Klassifikationsmodellen grundsätzlich vier mögliche Konzepte unterscheiden:
1. Local Classifier per Node (LCN): Dieses Konzept besteht darin, einen binären Klassifizierer für jede Kategorie und somit für jeden Knoten der IPC Hierarchie zu trainieren. Diese Herangehensweise resultiert in genauso vielen Klassifizierern, wie es Kategorien innerhalb der Taxonomie gibt.
2. Local Classifier per Parent Node (LCPN): Dieses Konzept besteht darin, auf Elternknotenebene Klassifizierer zu trainieren, welche vorhersagen, in welche  Kindknoten ein Dokument kategorisiert werden kann.
3. Local Classifier per Level (LCL): Bei diesem Konzept wird ein Klassifizierer für jedes Hierarchie-Level der IPC trainiert. Entsprechend müssen hier die Vorhersagen des nächsthöheren Levels verwendet werden, um die Zielmenge der Vorhersagen zu reduzieren.
4. Global Classifier (GC): Bei diesem Konzept wird ein Klassifizierer auf die gesamte IPC Hierarchie trainiert. Der Unterschied zur "flachen" Betrachtung der Hierarchie besteht hierbei darin, dass hierarchische Informationen trotzdem in die Vorhersage eingebunden werden.

Während der GC lediglich einen einzigen Klassifizierer benötigt, erfordern die anderen drei Konzepte je nach gewählter Klassifierungstiefe eine unterschiedliche Anzahl an einzelnen Klassifizierern wie in [Tabelle 2](#t2) dargestellt.


|Name  <a name="t2"></a>      |# Categories|# LCN |# LCPN |# LCL |
|:---------------------------:|:----------:|:--------------:|:---------------:|:--------------:|
|Section                      |8           |8               |1                |1               |    
|Class                        |129         |129             |8                |1               |
|Subclass                     |638         |638             |129              |1               |
|Main group                   |7391        |7391            |638              |1               |
|Subgroup                     |64046       |64046           |7391             |1               |


Das zweite für diese Seminararbeit relevante Merkmal des AHTC-Problems ist die Anforderung, Patente in mehr als eine Kategorie klassifizieren zu können. Dies ist gerade dann relevant, wenn eine Hierarchie modelliert wird, da hier für ein Patent mehrere Pfade durchlaufbar sein müssen.  

Das Ziel dieser Arbeit ist es, zwei Modelle zu entwickeln, die jeweils eines der vorgestellten hierarchischen Konzepte realisieren. Die Entscheidung fiel hierbei auf LCN und LCPN, da erwartungsgemäß bessere Klassifikationsergebnisse erzielbar sind durch Klassifikatoren, die auf eine kleinere Zielmenge, nämlich nur die für die Knoten relevante Zielmenge trainiert sind. Ein Training auf Levelebene bspw. birgt die Gefahr, dass Kategorien derselben Elternkategorie in der Zielfunktion näher beiander liegen und weniger gut unterschieden werden können, da der Klassifikator durch die im Zielraum weiter weg liegenden Kategorien anderer Elternkategorien einen makroskopischeren Fokus legt. Beide Klassifikationsmodelle sollten dem Anspruch gerecht werden, Multi-Labels klassifizieren zu können.

Zusätzlich zu diesen beiden Merkmalen des AHTC-Problems muss ebenfalls entschieden werden, bis zu welchem Level ein hierarchisches Klassifikationsmodell ausgelegt und getestet werden sollte. Unter Betrachtung der ab dem vierten Level (Main Group) sehr dünn werdenden Datensätzen für einige Kategorien erscheint es als sinnvoll, das hierarchische Klassifikationsmodell bis zum dritten Level Subclass zu testen und evaluieren. Dies deckt sich auch mit der Vorgehensweise eines Großteils möglicher Vergleichsarbeiten.


### C. Vergleichsmodelle und Evaluationsmetriken <a name="kapitel2.3"></a>

Das Ziel dieser Arbeit ist die Umsetzung eines AHTC Systems, das eine möglichst gute Performance im Vergleich zu einem einfachen Baseline-Modell sowie zu verschiedenen bereits existierenden AHTC Implementierungen liefert. Hierfür werden zwei verschiedene Hierarchie-Modelle in unterschiedlichen Varianten implementiert, die unten näher beschrieben werden.

Als Baseline-Modell dient ein einfacher, flacher Klassifizierer in Form eines Entscheidungsbaums. Dieser nutzt demnach keine der vorliegenden hierarchischen Informationen, sondern klassifiziert direkt auf dem Ziellevel innerhalb der Hierarchie. Um den Vergleich zwischen dem Baseline-Modell und den beiden Hierarchie-Modellen ziehen zu können, wird die F1-Metrik verwendet. Diese gewichtet die gegenläufige Maße Precision und Recall gleich gegeneinander und stellt somit sicher, dass das System sowohl eine möglichst niedrige Anzahl an False Negatives sowie False Positives produziert. Im Folgenden ist die F1-Metrik mit ihren Bestandteilen definiert:

<img src="img/f1_score.png" style="width: 300px;">

Gomez and Moans [[2014](#ref2)] liefern eine umfassende Übersicht und Gegenüberstellungen bestehender Arbeiten im Bezug auf die hierarchische Klassifizierung von Patenten. Darin wird deutlich, dass sich die existierenden Paper oftmals nur schwer miteinander vergleichen lassen. Dies liegt mitunter an der unterschiedlichen Wahl des zugrundeliegenden Datensatzes. Jedoch selbst bei der Wahl des gleichen Datensatzes ist ein Vergleich der Ergebnisse meistens nur schwer möglich, da entweder unterschiedliche Untermengen des Datensatzes, unterschiedliche Ziellevel für die Klassifikation oder unterschiedliche und teils eigenentwickelte Metriken gewählt wurden.

Um den Vergleich zu bereits existierenden Arbeiten ziehen zu können, wird für diese Arbeit auf die einzigen beiden Paper zurückgegriffen, die sowohl den ganzen WIPO-alpha Datensatz sowie einheitlich definierte Metriken verwenden: 
Fall et al. [[2003](#ref3)] entwickelten ein Modell unter Verwendung von u.a. Support Vector Machines und Tikk et al. [[2005](#ref4)] entwickelten ein Modell unter Verwendung eines UFEX (Universal Feature Extractor) Klassifizierers. Beide bedienten sich zur Evaluation ihrer Klassifikation auf Class sowie Subclass-Ebene drei spezieller Metriken, die auch für diese Arbeit im Sinne der Vergleichbarkeit zum Einsatz kommen und im folgenden nach Tikk et al. [[2005](#ref4)] definiert und in der darunter liegenden Abbildung visualiert werden:

1. Top Prediction Accuracy: Vergleich der top-vorhergesagten Kategorie mit der Haupt-IPC-Kategorie des Patents
2. Three Guesses Accuracy: Vergleich der drei top-vorhergesagten Kategorien mit der Haupt-IPC-Kategorie des Patents (erfolgreich bei einer Übereinstimmung)
3. All Classes Accuracy: Vergleich der top-vorhergesagten Kategorie mit allen IPC-Kategorien des Patents.

<img src="img/accuracy.png" style="width: 400px;">

Am Ende dieser Arbeit wird die Performance der beiden Hierarchie-Modelle anhand der vorgestellten Metriken gemessen. Diese wird im Anschluß den Performances des Baseline-Modells sowie der Vergleichsmodelle aus der Literatur gegenübergestellt.

# 3. Implementierung <a name="kapitel3"></a>

In diesem Abschnitt wird die Implementierung unseres AHTC Systems beschrieben und umgesetzt. Nach einem Einblick in den WIPO-alpha Datensatz wird auf das Vorgehen beim Preprocessing, die Implementierung der Hierarchie sowie der Umsetzung und den Variationen der beiden Hierarchie-Modelle eingegangen.

Wie aus dem folgenden Code-Abschnitt ersichtlich wird, liegt der eigentliche Code, insbesondere die eigentliche Hierarchie- und Modell-Logik, in sechs Python-Dateien, die in diesem Jupyter-Notebook importiert und benutzt werden. Alle implementierten Klassen und Methoden wurden nach geltenden Standards mittels Docstring beschrieben:

1. preprocessing.py: Enthält Methoden zum Extrahieren und Preprocessen der XML-Patent-Dokumente
2. data_model_handling.py: Enthält Methoden und Klassen zum Laden und Transformieren von Daten sowie zum Laden und Speichern von Modellen
3. hierarchy.py: Enthält Methoden zur systematischen Generierung der Hierarchie-Logik sowie zur Anwendung einer generierten Hierarchie zum Vorhersagen von Patent-Kategorien
4. lcpn.py: Enthält alle nötigen Klassen und Methoden zur Repräsentation eines Knoten innerhalb einer LCPN-Hierarchie
5. lcn.py: Enthält alle nötigen Klassen und Methoden zur Repräsentation eines Knoten innerhalb einer LCN-Hierarchie
6. scoring.py: Enthält alle Methoden zur Errechnung der oben dargestellten Metriken

In [ ]:
from preprocessing import preprocess
from data_model_handling import load_object, save_object, import_data, get_level_data
import lcn
import lcpn
from hierarchy import create_classification_hierarchy, match_predictions, predict_with_hierarchy
from scoring import print_f1_score, print_accuracy_scores_top_ipc, print_accuracy_score_all_ipcs

import pandas as pd
import numpy as np
import math
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

### A. Daten <a name="kapitel3.1"></a>

Bei den für diese Seminararbeit genutzten Daten handelt es sich um die Patentanmeldungen des [WIPO-en-alpha (2002)](https://www.wipo.int/classifications/ipc/en/ITsupport/Categorization/dataset/index.html) Datensatzes. Dieser beinhaltet etwa 75.000 Patentdokumente, welche zu Forschungszwecken von der WIPO zur Verfügung gestellt werden. 

Die Patentdokumente sind innerhalb einer verschachtelten Ordnerstruktur als xml Dateien gespeichert. Diese Dateien beinhalten u.a. die Patentansprüche, Namen der Erfinder, Titel, Zusammenfassungen und Patenttexte sowie IPC-Codes der Patentanmeldungen. 

Die für einen textbasierten Klassifikationsansatz potenziell relevanten Abschnitte für den Input sind der Titel, die Zusammenfassung, der Patenttext sowie die Patentansprüche. Anhand der [Tabelle 3](#t3) lässt sich erkennen, dass der Patenttext mit Abstand die meisten einzigartigen Wörter aufweist und somit als ergiebigste Quelle im Hinblick auf die Unterscheidbarkeit der Dokumente erscheint. Weiterhin ist für die anderen Abschnitte diesem gegenüber eine hohe Redundanz zu erwarten. 

| Abschnitt<a name="t3"></a> | Dursch. einzigartige Wörter |
|:--------------------------:|:---------------------------:|
|Titel                       |5.2                          |                     
|Zusammenfassung             |36.1                         |                    
|Patenttext                  |747.3                        |                   
|Patentansprüche             |103.8                        |    

Entprechend dieser Erkenntnis werden im weiteren Vorgehen der Patenttext als Input sowie die IPC-Codes als Output extrahiert und verarbeitet. Bezüglich der IPC-Codes ist hierbei zu erwähnen, dass jedes Patent einen Haupt-IPC-Code und möglicherweise zusätzliche Neben-IPC-Codes besitzt. Dies ist insbesondere für die Errechnung der bereits vorgestellten Evaluationsmetriken relevant.

Basierend auf den Haupt-IPCs auf dem ersten Hierarchie-Level (section) der Patente findet sich im Folgenden eine Visualisierung der ausgewählten Daten anhand einer Dimensionsreduzierung mit einer Single Value Decomposition auf zwei Dimensionen. Diese zeigt deutlich auf, dass die einzelnen Kategorien sich teils wesentlich in ihren Patenttext-Daten zu unterscheiden scheinen. Somit erfüllen sie zumindest auf diesem Hierarchie-Level eine gute Voraussetzung für eine Klassifikation. Zusätzlich lässt sich in der Legende an der in den eckigen Klammern dargestellten Anzahl der Dokumente pro Section im Trainingsdatensatz erkennen, dass der Datensatz teils sehr unbalanciert ist.

<img src="img/data.png" style="width: 600px;">

### B. Preprocessing <a name="preprocessing"></a>

Das Preprocessing umfasst die Extraction, Bereinigung und Vorbereitung jedes einzelnen Patents in den bereits im Datensatz vordefinierten Trainings- und Testdatensätzen. Hierfür wird jede einzelne Patent-XML-Datei in den Rohdaten durchlaufen, die IPC-Codes sowie der Patenttext werden extrahiert, der Patenttext wird durch verschiedene Preprocessing-Schritte bereinigt und abschließend werden IPC-Codes und der bereinigte Patenttext in einer txt-Datei entsprechend der originalen Ordnerstruktur in einem neuen Zielverzeichnis abgelegt.

Das explizite Preprocessing des Patenttextes umfasst dabei mehrere Schritte, die insbesondere dazu dienen den Text zu standardisieren sowie in seiner Komplexität und seinem Umfang zu reduzieren ohne dabei wesentlichen Informationsgehalt zu verlieren, der für die Unterscheidbarkeit sowie Zuordenbarkeit der Dokumente von Relevanz ist. Die einzelnen Schritte des Preprocessings sind im Folgenden beschrieben.

Zunächst werden alle Kontraktionen im Text in volle Worte umgewandelt, um bessere Voraussetzungen für das anschließende Tokenizing zu schaffen, bei dem der Text in eine Liste von Tokens gewandelt wird, welche einfacher zu handhaben sind. Als Token versteht man eine Zeichenkette, welche im Text durch zwei Leerzeichen gekennzeichnet ist. 

Die Token-Liste wird anschließend von allen Zahlen befreit, da Zahlen ohne ihren unmittelbaren kontextuellen Zusammenhang keinen Informationsgehalt erwarten lassen. Daraufhin werden Stoppwörter aus der Liste an Token entfernt. Stoppwörter sind hierbei Wörter welche sehr häufig innerhalb einer Sprache auftauchen und einem Text kein zusätzlichen Informationsgehalt hinsichtlich der Unterscheidbarkeit zu anderen Texten bieten. Um unsere Token-Liste speziell auf die Patent-Thematik hin zu optimieren, wird neben einer standardmäßigen Stoppwörter-Liste mit Stoppwörtern der englischen Sprache auch eine Stoppwörter-Liste des US Patent und Trademark Office genutzt. 

Nachdem ebenfalls Satzzeichen und Sonderzeichen aus den Token entfernt wurden und alle verbleibenden Token in kleinbuchstäbige Token vereinheitlich wurden, findet mit der Lemmatisierierung ein elementarer Schritt zur Informationsverdichtung statt. Ziel dieser Textnormalisierungstechnik ist es Flexionsformen von Wörtern auf ihre Grundform zu reduzieren. Im Gegensatz zur ähnlichen Stemming Technik werden die Wortart der Flexion und der Kontext, in dem sie steht, beachtet. Beim Stemming hingegen wird versucht Worte allein durch das Abschneiden von Wortteilen auf ihre Stammform zu reduzieren. Die Lemmatisierung resultiert dabei im Gegensatz zur Stemming Technik allgemein in einer stabileren Normalisierung, wie unterschiedliche Untersuchungen wie beispielsweise durch Balakrishnan & Lloyd-Yemoh [[2014](#ref5)] zeigen konnten. Die Unterschiede der Techniken werden in [Tabelle 4](#t4) anhand verschiedener Flexionsformen und deren Interpretation durch einen Lemmatisierer und einen Stemmer verdeutlicht.

| Wort<a name="t4"></a>      |Lemma (Wordnet Lemmatizer) |Stem (Porterstemmer)    |
|:--------------------------:|:-------------------------:|:----------------------:|
|meeting (Nomen)             |meeting                    |meet                    |               
|met (Verb)                  |meet                       |met                     |                  
|meeting (Verb)              |meet                       |meet                    |                
|meet (Verb)                 |meet                       |meet                    |

Unter den verfügbaren Lemmatizern fiel die Entscheidung auf den Wordnet-Lemmatizer, der auf der großen lexikalischen Datenbank "Wordnet" basiert und über das NLTK-Modul-Interface direkt zugänglich ist. In eigens durchgeführten Tests erzielte dieser Lemmatizer durch den Einbezug von Part-of-Speach-Tags für die einzelnen Worte im Vergleich zu anderen Lemmatizer-Implementierungen wie den TextBlob Lemmatizer or Standford CoreNLP die stabilisten Ergebnisse.

Im Folgenden Code findet zunächst die Definition des Quell- und Zielverzeichnises statt sowie eine Auflistung aller Dateien, die fehlerhafte Inhalte aufwiesen, die keine eindeutige Verarbeitung zulassen, und deshalb im Preprocessing übergangen und nicht weiterverwendet werden. Anschließend wird das Preprocessing ausgeführt, wobei das Wort-Limit aus im Hinblick auf die späteren Modelle aus Performance und Rechenkapazitätsgründen auf 300 gesetzt wird.

In [ ]:
directory = '/home/ubuntu/Jupyter Notebooks/dataset/unpreprocessed/'
clean_directory = '/home/ubuntu/Jupyter Notebooks/dataset/preprocessed/'
path_split = 'unpreprocessed/'

faulty_files = ['SE0001854_05042001.xml', 'US0011521_07122000.xml',
                'IB9901525_16032000.xml', 'EP0012293_06062002.xml',
                'US0021074_15022001.xml', 'IT0000317_08022001.xml',
                'EP0010366_10052001.xml', 'ZA0000176_05042001.xml', 
                'US0106001_30082001.xml', 'AU0000791_11012001.xml']

preprocess_directory(directory, path_split, faulty_files, clean_directory, 300)

### C. Hierarchie-Implementierung <a name="kapitel3.3"></a>

Für die Implementierung der Hierarchie wurde ein Design gewählt, das als generelles Framework für hierarchische, knotenbasierte (LCN oder LCPN) Multilabel-Klassifizierung für jedes Ziel-Hierarchie-Level genutzt werden kann. 

Knoten innerhalb der Hierarchie werden durch eine ParentNode-Klasse repräsentiert, die für jede Hierarchie-Strategie (LCN oder LCPN) eigens in den Python-Dateien lcn.py und lcpn.py implementiert ist. Ein Objekt dieser ParentNode-Klasse speichert verschiedene Informationen:

1. Namen und das Level des Knotens
2. Daten-Transformer-Objekte für Input-(X) und Labeldaten(y)
3. Das Classifier-Modell für LCPN bzw. eine Liste mit den Child-Classifier-Modellen für LCN
4. Eine Liste mit den Namen aller Zielklassen aus y
5. Eine Liste mit Referenzen auf alle Child-Knoten

Die Generierung der Hierarchie beginnt bei dem Seed-Knoten. Bei der Generierung eines Knotens werden die Rohdaten zunächst auf die entsprechende Untermenge reduziert (beim Seed-Knoten nicht nötig) und die Labels in ihrer Länge an das Level angepasst. Danach erfolgt die Vektorisierung und Dimensionsreduzierung der Textdaten (x) sowie die Vektorisierung der Labeldaten (y). Die entsprechenden auf die Trainingsdaten angepassten Transformer-Objekte (TransformerPipeline bestehend aus CountVectorizer, TfidfTransformer und TruncatedSVD für X sowie CountVectorizer für y) werden im Knoten gespeichert, um später auf ungesehene Daten angewandt werden zu können. Zudem wird eine Liste mit den Namen aller Zielklassen aus y gespeichert. Anschließend wird das Classifier-Modell (für LCPN) bzw. die Child-Klassifier-Modelle (für LCN) trainiert und ebenfalls im Knoten gespeichert. Die Liste mit den Referenzen auf Child-Knoten wird zunächst leer gelassen. 

Ist der Seed-Knoten erstellt, wird dessen Liste mit den Namen aller Zielklassen durch-iteriert. Für jede Zielklasse wird ein neuer Knoten erstellt, die relevante Untermenge der Daten lässt sich dabei durch ein String-Match zwischen Namen und den Labels in den Daten finden. Jeder so erstellte Child-Knoten wird anschließend in der Child-Knoten-Liste des Parent-Knotens (in diesem Fall der Seed-Knoten) gespeichert. Dieses iterative Vorgehen wird bis zum gewählten Ziel-Hierarchie-Level fortgeführt, wobei nach Erstellung eines Knotens immer direkt seine Child-Knoten erstellt werden. 

Dieses Design der Hierarchie hat nicht nur den Vorteil, dass die gesamte Hierarchie anschließend lediglich durch eine Referenz auf den Seed-Knoten zugänglich ist, sondern erweist sich auch als vorteilhaft bei der Umsetzung der hierarchischen Vorhersage. Denn ebenso wie bei der iterativen Generierung von oben nach unten, lässt sich nun die Gesamtvorhersage auf dem Ziel-Hierarchie-Level iterativ von oben nach unten erzeugen. Anhand der Parent-Child-Referenz-Struktur wird jeder einzelne Knoten mit seinem trainierten Modell durchlaufen und erzeugt eine Vorhersage auf dem ihm übergebenen Input. Für das Erhalten einer Gesamtvorhersage für das Ziel-Hierarchie-Level durch die Hierarchie hindurch ergeben sich jedoch zwei Problemstellungen:

1. Die erste Problemstellung bezieht sich auf die Weitergabe der Informationen bzw. Vorhersagen zwischen den Leveln. Für den Fall, dass die Vorhersagen auf jedem Level strikt binär ausfallen (1: Patent gehört zu der Klasse, 0: Patent gehört nicht zur der Klasse) muss sichergestellt werden, dass der Child-Knoten nur noch diejenigen Dokumente berücksichtigt, die vom Parent-Knoten mit 1 für die Parent-Klasse gelabelt wurden. Für den Fall einer probalistischen Vorhersage (Werte stetig zwischen 0 und 1) sollten die Vorhersagen des Child-Knoten entsprechend der Vorhersage des Parent-Knoten für die Parent-Klasse gewichtet werden. Die hier gewählte Lösung dieser Problemstellung erfordert die Vorhersage durch jeden Knoten auf allen Dokumenten des entsprechenden Test-Datensatzes, zunächst ohne in irgendeinerweise die Vorhersage des Parent-Knotens miteinzubeziehen. Die Parent-Child-Referenz-Struktur erlaubt es nun, den Namen des Child-Knoten mit der Child-Knoten-Liste des Parent-Knoten zu matchen, um so den Index der Spalte zu erhalten, die die Vorhersage für die Parent-Klasse beinhaltet. Diese Spalte wird abschließend in einer Matrix-Multiplikation mit der Vorhersage des Child-Knoten multipliziert und setzt somit alle für die Parent-Klasse als 0 gelabelten Dokumente auch in der Child-Knoten-Vorhersage auf 0 (für binäre Vorhersagen) bzw. gewichtet die Child-Knoten-Vorhersage durch die Vorhersage für die Parent-Klasse wie. Zur Veranschaulichung ist dies in folgender Abbildung exemplarisch für ein Matching des Child-Knoten A mit der Vorhersage des Seed-Knotens dargestellt.

<img src="img/hierarchy1.png" style="width: 600px;">

2. Die zweite Problemstellung betrifft die Tatsache, dass aufgrund der Vielzahl von Knoten auf dem Ziel-Hierarchie-Level (sofern es nicht direkt das erste Level ist) auch die Vorhersagen in vielen einzelnen Stücken vorliegen und für jedes Dokument zu einer Gesamtvorhersage auf dem Ziel-Hierarchie-Level so zusammengefasst werden müssen, dass sie anschließend mit den wahren Daten verglichen werden können. Die in der Implementierung gewählte Lösung bedient sich hierfür der bereits vektorisierten Label-Testdaten (y) zusammen mit einer Liste, die den einzelnen Spalten die entsprechenden Namen der Zielklasse zuordnet. Entsprechend der Form der Label-Testdaten-Matrix wird eine Null-Matrix erstellt als Ergebnismatrix initiiert. Dies erlaubt es nun, für jeden Knoten, der sich in dem Ziel-Hierarchie-Level befindet, ein Matching über die Zielklassen-Liste durchzuführen und so die finalen Vorhersagewerte suksessive in die Spalten der Ergebnismatrix zu addieren. Verdeutlicht wird dies in folgender Abbildung, in der exemplarisch das Updaten der Ergebnismatrix auf Level 3 der Hierarchie durch die Vorherhersage des Knotens H05 dargestellt ist.

<img src="img/hierarchy2.png" style="width: 600px;">

Für beide Designs (LCN und LCPN) erzeugen die Klassifizierer standardmäßig in jedem Knoten probalistische Vorhersagen, die stetig zwischen 0 und 1 liegen. Um zwischen probalistischer und binärer (diskret entweder 0 oder 1) Vorhersage in der kompletten Hierarchie wechseln zu können, lässt sich der entsprechenden Methode ein boolescher Parameter  namens 'binary_pred' übergeben. Ist der Wert dieses Paramters 'wahr', werden alle Vorhersagen anhand des Grenzwertes 0.5 in diskrete binäre Vorhersagen transformiert. Dies ist nötig, damit alle oben beschriebenen Metriken berechnet werden können.

### D. Modelle <a name="kapitel3.4"></a>

In diesem Abschnitt werden die entwickelten Modelle mit ihren verschiedenen Variationen beschrieben und umgesetzt. Zunächst findet sich hier die Implementierung des Baseline-Modells (a) anschließend die Implementierung der beiden Hierarchie-Modelle (b und c).

Zunächst werden die Verzeichnis-Pfade der preprozessierten Daten definiert.

In [ ]:
train_path = '/home/ubuntu/Jupyter Notebooks/dataset/preprocessed/train/'
test_path = '/home/ubuntu/Jupyter Notebooks/dataset/preprocessed/test/'

#### a. Baseline Modell: Flat, Decision Tree <a name="kapitel3.4.1"></a>

Das Baseline-Modell wurde als ein sehr simples, "flaches" Decision-Tree-Modell gewählt, welches keine hierarchischen Informationen miteinbezieht. Dementsprechend wurde hier auch nicht auf die Hierarchie-Implementierungen zurückgegriffen, sondern abgesehen vom Laden der Daten alles direkt basierend auf den zur Verfügung stehenden sklearn-Klassen im Jupyter-Notebook gecoded.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

levels = 3
raw_train_data = import_data(train_path, levels)
df_train = get_level_data(raw_train_data, levels, None)

X_train = df_train['document']
count_vec_y = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, token_pattern = r"(?u)\b\w+\b")
y_train = count_vec_y.fit_transform(df_train['ipcs'])

clf_pipeline = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('svd', TruncatedSVD(n_components = 100)), ('clf', DecisionTreeClassifier(max_leaf_nodes=2000))])
clf_pipeline.fit(X_train, y_train.toarray())
print("BASELINE MODEL TRAINED.")

raw_test_data = import_data(test_path, levels)
df_test = get_level_data(raw_test_data, levels, None)
X_test = df_test['document']
y_test = count_vec_y.transform(df_test['ipcs'])

# PREDICTION ON TRAIN SET
y_pred_train = clf_pipeline.predict(X_train)
#print_f1_score(y_train, y_pred_train, "micro")

# PREDICTION ON TEST SET
y_pred_test = clf_pipeline.predict(X_test)
#print_f1_score(y_test, y_pred_test, "micro")

#### b. Hierarchie Modell 1: LCPN, Neuronale Netze <a name="kapitel3.4.2"></a>

Das Hierarchie-Modell 1 beruht auf dem LCPN-Design und nutzt Neuronale Netze als Classifier für jeden Knoten. 

Neuronale Netze bieten sich in diesem Anwendungsfall an, da sie im Vergleich zu anderen Machine-Learning-Methoden besonders gut mit großen Datenmengen und insbesondere einer hohen Zahl an Datendimensionen (Features) umgehen können. Darüber hinaus handelt es sich beim LCPN-Design in jedem Knoten um eine Multilabel-Klassifikation, entsprechend liegt der gewünschte Output für jede Ziel-Klasse zwischen 0 und 1 und soll die Wahrscheinlichkeit dafür ausdrücken, dass das entsprechende Label für das Patent passend ist. Dies lässt sich in einem neuronalen Netz mithilfe der 'Sigmoid'-Aktivierungsfunktion im Output-Layer realisieren, welche für jedes Output-Neuron einen Wert zwischen 0 und 1 produziert, dessen Größe unabhängig von den anderen Output-Werten ist. Weiterhin ist die gewählte Loss-Funktion 'binary_crossentropy' sehr gut für die Multi-Label-Problemstellung ausgelegt.

Für das Hierarchie Modell 1 wurden verschiedene Netzdesigns mit unterschiedlichen Anzahl an Layern, unterschiedlicher Anzahl an Neuronen pro Layer sowie unterschiedlichen Aktivierungsfunktionen in den Hidden Layern ausprobiert. Desweiteren wurden verschiedene Parameter-Kombinationen getestet, u.a. die Lernrate des gewählten Adam-Optimizers, die Anzahl der Epochen sowie der Gedulds-Wert für die Early-Stopping-Regel. Das anhand dieser Tests am Ende gewählte Design lässt sich in der Datei lcpn.py finden. Für die finalen Ergebnisse wurde ein Netzdesign mit zwei Hidden-Dense-Layern (600 Neuronen) sowie zwei dazwischen liegenden Dropout-Layern (Dropout-Faktor 0.2) gewählt.

Im folgenden Code lassen sich zunächst drei Paremeter festlegen:
- 'levels': Bestimmt, bis zu welchem Level die Hierarchie generiert werden soll (wobei 1 das oberste Level darstellt) und auch entsprechend vorhersagen soll. 
- 'only_main_ipc': Bestimmt, ob für die gewählten Trainings- und Testdaten nur die Haupt-IPC oder alle IPCs geladen werden sollen
- 'binary_pred': Bestimmt, ob die Vorhersage in jedem Knoten der Hierarchie binär oder probalistisch erfolgen soll.


Die Wahl der letzten beiden Parameter muss davon abhängig gemacht werden, für welche Metrik ein optimales Ergebnis erzielt werden soll:
- F1-Score: Dieser Score berücksichtigt alle IPCs. Dementsprechend sollten sowohl für das Training der Hierarchie alle IPCs geladen werden. Desweiteren benötigt dieser binäre Vorhersagen für jede Zielklasse. Dementsprechend lautet die Wahl der beiden Paramter hier {'False', 'True'}.
- Top Prediction Accuracy & Three Guesses Accuracy: Diese beiden Scores vergleichen die Top- bzw. die top 3 Vorhersagen gegen die Haupt-IPC. Entsprechend sollten für das Training der Hierarchie auch nur die Haupt-IPC der Patent geladen werden, da die zusätzlichen IPCs vom gewünschten Zielwert weglenken würden. Um die Top- bzw. die top 3 Vorhersagen bestimmen zu können sollten innerhalb der Hierarchie nur probalistische Vorhersagen produziert werden. Folglich lautet die Wahl der beiden Paramter hier {'True', 'False'}
- All Classes Accuracy: Dieser Score vergleicht die Top-Vorhersage mit allen IPCs eines Patents. Dementsprechend sollten für das Training der Hierachie alle IPCs geladen werden. Um die Top-Vorhersage bestimmen zu können sollten innerhalb der Hierarchie nur probalistische Vorhersagen produziert werden. Folglich lautet die Wahl der beiden Paramter hier {'False', 'False'}

Entsprechend der Wahl des Scores bzw. der Parameter muss die jeweilige Scoring-Methode im Code aktiviert (vom Kommentar-Hashtag befreit) werden.

Nach der Wahl der Parameter wird die Hierarchie zunächst generiert und trainiert, anschließend lässt sie sich ggfs. unter dem angebenen Zielpfad abspeichern oder, falls bereits abgespeichert direkt laden. Anschließend folgen die Vorhersagen auf den Trainings- und Testdaten sowie die Errechnung der gewählten Metrik.

In [ ]:
levels = 1
only_main_ipc = False
binary_pred = False

# CREATE HIERARCHY
lcpn_seed_node = create_classification_hierarchy(train_path, levels, lcpn, only_main_ipc)

# SAVE OR LOAD HIERARCHY
#target_path = '/home/ubuntu/Jupyter Notebooks/models/'
#save_object(lcpn_seed_node, target_path + "Model1_Lv" + str(levels) + '.pkl')
#lcpn_seed_node = load_object(target_path + "Model1_Lv"+ str(levels) + ".pkl")

# PREDICTION ON TRAIN SET
y_pred_train, y_train = predict_with_hierarchy(lcpn_seed_node, levels, train_path, only_main_ipc, binary_pred)
#print_f1_score(y_train, y_pred_train, "micro")
#print_accuracy_scores_top_ipc(y_train, y_pred_train)
#print_accuracy_score_all_ipcs(y_train, y_pred_train)

# PREDICTION ON TEST SET
y_pred_test, y_test = predict_with_hierarchy(lcpn_seed_node, levels, test_path, only_main_ipc, binary_pred)
#print_f1_score(y_test, y_pred_test, "micro")
#print_accuracy_scores_top_ipc(y_test, y_pred_test)
#print_accuracy_score_all_ipcs(y_test, y_pred_test)

#### c. Hierarchie Modell 2: LCN, SGD-Classifier<a name="kapitel3.4.3"></a>

Das Hierarchie-Modell 2 beruht auf dem LCN-Design und nutzt für die finalen Ergebnisse Stochastic Gradient Descent (SGD) Classifier sowohl mit einer Hinge-Loss-Funktion (entspricht einer linearen Support Vector Machine) wie auch mit einer Log-Loss-Funktion als binäre Classifier für die jeweiligen Zielklassen, deren Vorhersage auf Parent-Knoten-Ebene für jeden Knoten zusammengefasst werden. 

Für dieses Modell wurden sowohl kernelbasierte Support Vector Machines, Naive Bayes als auch Stochastic Gradient Descent Classifier mit verschiedenen Loss-Funktionen getestet. Die kernelbasierten Support Vector Machines wurden aufgrund hoher Operationskomplexität und damit extremen Laufzeiten ausgeschlossen. Unter den restlichen Optionen konnte der Stochastic Gradient Descent Classifier mit einer Hinge-Loss Funktion bzw. einer Log-Loss Funktion die besten Resultate erzielen. Wie bei einem normalen Gradient Descent (GD) Algorithmus werden beim Stochastic Gradient Descent (SGD) Verfahren ebenfalls iterativ Parameter angepasst um eine gegebene Loss-Funktion zu minimieren. Allerdings findet beim SGD die Aktualisierung der Parameter nicht erst nach dem Durchlauf des kompletten Datensatzes, sondern nach jedem einzelnen Sample statt, was zu wesentlich effizienteren Laufzeiten führt bei großen Datensätzen wie unserem führt.

Entsprechend der Wahl des Scores bzw. der Parameter muss wie beim vorherigen Modell die jeweilige Scoring-Methode im Code aktiviert (vom Kommentar-Hashtag befreit) werden.

In [ ]:
levels = 1
only_main_ipc = False
binary_pred = False

# CREATE HIERARCHY
lcn_seed_node = create_classification_hierarchy(train_path, levels, lcn, only_main_ipc)

# SAVE OR LOAD HIERARCHY
#target_path = '/home/ubuntu/Jupyter Notebooks/models/'
#save_object(lcpn_seed_node, target_path + "Model1_Lv" + str(levels) + '.pkl')
#lcpn_seed_node = load_object(target_path + "Model1_Lv"+ str(levels) + ".pkl")

# PREDICTION ON TRAIN SET
y_pred_train, y_train = predict_with_hierarchy(lcn_seed_node, levels, train_path, only_main_ipc, binary_pred)
#print_f1_score(y_train, y_pred_train, "micro")
#print_accuracy_scores_top_ipc(y_train, y_pred_train)
#print_accuracy_score_all_ipcs(y_train, y_pred_train)

# PREDICTION ON TEST SET
y_pred_test, y_test = predict_with_hierarchy(lcn_seed_node, levels, test_path, only_main_ipc, binary_pred)
#print_f1_score(y_test, y_pred_test, "micro")
#print_accuracy_scores_top_ipc(y_test, y_pred_test)
#print_accuracy_score_all_ipcs(y_test, y_pred_test)

### E. Ergebnisse <a name="kapitel3.5"></a>

Im folgenden werden die resultierenden Ergebnisse vorgestellt. Zuächst werden die beiden entwickelten Hierarchie-Modelle anhand des F1-Scores mit dem "flachen" Baseline-Modell verglichen. Anschließend findet ein Vergleich der beiden Modelle mit den bereits beschriebenen Modellen von Fall et al. [[2003](#ref3)] sowie Tikk et al. [[2005](#ref4)] anhand der in diesen Papern benutzten Accuracy-Metriken statt. Für die Generierung der Modelle wurden die ersten 300 Worte des preprozessierten Patenttextes für jedes Patent als Input betrachtet.

**Vergleich Modelle vs. Baseline-Modell (F1-Score)**

|Level<a name="t5"></a>     |Baseline            |Model 1 (LCPN-NN)   |Model 2 (LCN-SGD)  |  
|:-------------------------:|:------------------:|:------------------:|:-----------------:| 
|Section                    |        0.62        |         0.73       |     0.69          |
|Class                      |        0.41        |         0.60       |     0.57          |
|Subclass                   |        0.24        |         0.47       |     0.42          |

Der Vergleich zum Baseline-Modell zeigt eine deutliche Steigerung des Klassifikationserfolgs durch die beiden entwickelten Modelle. Auffällig ist hierbei insbesondere, dass die Überlegenheit größer wird je tiefer man sich in der Hierarchie bewegt und damit auch je größer die Komplexität der Klassifikationsaufgabe wird.

**Vergleich Modelle vs. Paper-Modelle (Accuracy-Scores)**

Metrik 1: Accuracy Top Prediction

|Level<a name="t6"></a>     |Ref-Model 1 (Fall)  |Ref-Model 2 (Tikk) |Model 1 (LCPN-NN) |Model 2 (LCN-SGD) |  
|:-------------------------:|:------------------:|:-----------------:|:----------------:|:----------------:| 
|Class                      |        0.55        |        0.66       |        0.60      |        0.50      |
|Subclass                   |        0.41        |        0.55       |        0.45      |        0.41      |

Metrik 2: Accuracy Three Guesses

|Level<a name="t6"></a>     |Ref-Model 1 (Fall)  |Ref-Model 2 (Tikk) |Model 1 (LCPN-NN)   |Model 2 (LCN-SGD)  |  
|:-------------------------:|:------------------:|:------------------:|:------------------:|:------------------:| 
|Class                      |        0.79        |        0.89        |        0.82        |        0.79        |
|Subclass                   |        0.62        |        0.79        |        0.68        |        0.67        |

Metrik 3: Accuracy All IPCs

|Level<a name="t6"></a>     |Ref-Model 1 (Fall)  |Ref-Model 2 (Tikk)  |Model 1 (LCPN-NN)   |Model 2 (LCN-SGD)   |  
|:-------------------------:|:------------------:|:------------------:|:------------------:|:------------------:| 
|Class                      |        0.63        |        0.76        |        0.67        |        0.62        |
|Subclass                   |        0.48        |        0.66        |        0.57        |        0.41        |

Der Vergleich zu den Referenz-Modellen von Fall et al. [[2003](#ref3)] sowie Tikk et al. [[2005](#ref4)] zeigt hingegen ein gemischtes Bild. Model 2 schafft es nur für die Metrik Accuracy Three Guesses das Modell von Fall et al. [[2003](#ref3)] zu übertreffen und bleibt in allen anderen Vergleichen hinter der Performance beider Referenz-Modelle zurück. Das Model 1 hingegen zeigt durchweg eine bessere Performance und liegt für alle Metriken über dem Referenz-Modell von Fall et al. [[2003](#ref3)], bleibt jedoch ebenfalls hinter der Performance des Referenz-Modells von Tikk et al. [[2005](#ref4)]. Eine mögliche Erklärung für die dargestellte Performance von Tikk et al. [[2005](#ref4)] könnte jedoch die im Paper dargestellte Verwendung von Grenzwerten bei der Klassifikation um Missklassifikation auf den oberen Leveln der Hierarchie zu vermeiden. Diese bewirken demnach einen Stopp der Klassifikation auf dem jeweiligen Level sollte der verwendete Konformitätswert für alle möglichen Kategorien den Grenzwert unterschreiten [[Tikk et al., 2005](#ref4)]. Dabei wird nicht eindeutig dargestellt, in wie weit diese nicht zu Ende geführten Klassifikation bestimmter Dokumente in die Berechnung der verwendeten Metriken mit einfließt. Unter Verwendung eines solchen Mechanismus wäre auch für unsere Modelle eine Steigerung der Klassifikationsleistung zu erwarten.

## 4. Limitationen <a name="kapitel4"></a>

Im Folgenden wird auf die wesentlichen Limitationen eingegangen, denen die vorliegende Seminararbeit und damit insbesondere die implementierten Modelle unterliegen:
- Es wurden nur zwei der möglichen vier hierarchischen Konzepte aufgrund des limitierten Zeitrahmens und basierend auf entsprechend getroffenen Annahmen betrachtet. Es ist möglich, dass die Konzepte LCL und GC ebenfalls gute oder möglicherweise bessere Resultate zulassen.
- Die Modelle (Klassifizierer) innerhalb der Knoten konnten nur in begrenztem Umfang optimiert werden:
    - Neuronales Netz: Hier wurde nur eine begrenzte Anzahl möglicher Designs ausgetestet. Es wäre möglich, dass durch andere Designs mit anderen Arten von Layern (bspw. Convolutional) und anderen Activierungsfunktionen sowie anders gewählten Meta-Parametern verbesserte Ergebnisse erzielt werden können
    - Linearer Classificator: Hier wurden verschiedene Klassifikatoren getestet. Kernel-basierte Support Vector Machines ließen sich jedoch nur in begrenztem Umfang testen, da sie durch die hohe Operationskomplexität, die zwischen O(n_features x n_samples^2) und O(n_features x n_samples^3) liegt, und somit in einem Hierarchie-System mit einer Vielzahl solcher Klassifiziern zu extremen Laufzeiten führt.
- Bei der Entwicklung und Implementierung des Hierarchie-Frameworks wurde aus Komplexitätsgründen festgelegt, dass jeder Knoten innerhalb einer Hierarchie auf die gleiche Grundart an Klassifizierer zurückgreift (jeder Knoten neuronales Netz, SVM, Logistische Regression, etc.). Das Zulassen unterschiedlicher Klassifizierungsmodelle für unterschiedliche Knoten könnte jedoch zu einem besseren Fit auf die Daten beitragen und müsste weiter untersucht werden.
- Basierend auf dem größten erwarteten Informationsgehalt wurde lediglich der Patenttext als textuelle Inputdaten aus den Patenten einbezogen. Es ist jedoch nicht auszuschließen, dass andere Abschnitte innerhalb des Patent nicht trotzdem einen zusätzlichen Informationsgehalt liefern und zur Verbesserung der Resultate beitragen können.
- Der Vergleich mit anderen Arbeiten gestaltete sich in diesem Anwendungsfeld als besonders schwierig. Dies ist insbesondere darin begründet, dass die wenigsten Arbeiten ihren Fokus auf die Vergleichbarkeit zu anderen Arbeiten gelegt haben und durchweg unterschiedliche Evaluationsmetriken, unterschiedliche Datensätze oder Teildatensätze genutzt wurden. Um dennoch einen sinnvollen Vergleich zu existierenden Arbeiten ziehen zu können wurden als Vergleichsarbeiten nur jene berücksichtigt, die auf dem gleichen Datensatz, auf den gleichen Hierarchie-Leveln sowie anhand der gleichen Evaluationsmetrik gearbeitet haben. Durch diese Tatsache bedingt lässt sich jedoch nicht feststellen, wie die entwickelten Hierarchie-Modelle bezüglich dieser Metriken verglichen mit den anderen Arbeiten performen.

Die aufgelisteten Punkte liefern mögliche Anknüpfstellen, um auf den Ergebnissen und Implementierungen dieser Arbeit aufzubauen und die erzielten Resultate in zukünftigen Arbeiten weiter zu verbessern.


## 5. Fazit <a name="kapitel5"></a>

In dieser Seminararbeit wurde das Automated Hierarchical Text Classification Problem und seine Manifestierung im Bereich der Patentklassifikation dargestellt. Anhand einer theoretischen Einführung wurden mögliche Ansätze und zu berücksichtigende Anforderungen für die Entwicklung eines hierarchischen Klassifizierungsmodells vorgestellt und ein klares Vorgehen für die Entwicklung, Implementierung und Evaluierung eigener Modelle erarbeitet. Basierend auf dieser Grundlage konnten sowohl ein generelles Framework für eine knotenbasiertes hierarchisches Klassifizierungssystem sowie zwei darauf aufbauende explizite Modelle entwickelt und implementiert werden. 



Basierend auf diesen Ergebnissen und den im vorherigen Kapitel dargestellten Verbesserungsmöglichkeiten stellt diese Arbeit einen solide Grundlage für weitere Forschung, Entwicklungen und Applikationen im Bereich der automatischen hierarchischen Patentklassifikation dar.

## 6. Referenzen <a name="referenzen"></a>

[1] Silla Jr., C.N., Freitas, A.A., 2011. A survey of hierarchical classification across different
application domains. Data Mining and Knowledge Discovery 22(1-2), 31–72<a name="ref1"></a>

[2] Gomez, J.C. and Moens, M.F., 2014. A survey of automated hierarchical classification of patents. In Professional Search in the Modern World (pp. 215-249). Springer, Cham.<a name="ref2"></a>

[3] Fall, C.J., Törcsvári, A., Benzineb, K., Karetka, G., 2003. Automated categorization in the international patent classification. SIGIR Forum 37(1), 10–25<a name="ref3"></a>

[4] Tikk, D., Biro, G., Yang, J., 2005. Experiment with a hierarchical text categorization method on WIPO patent collections. In: Attoh-Okine, N., Ayyub, B. (eds.) Applied Research in Uncertainty Modeling and Analysis. International Series in Intelligent Technologies, vol. 20, pp. 283–302. Springer<a name="ref4"></a>

[5] Balakrishnan, V. and Lloyd-Yemoh, E., 2014. Stemming and lemmatization: a comparison of retrieval performances.<a name="ref5"></a>